# Assignment 2: Memory Task List Generation
## Computational Methods in Psychology and Neuroscience
### Psychology 4215/7215 --- Fall 2023

# Objectives

Upon completion of this assignment, the student will have:

1. Read in a stimulus pool from a file.

2. Created unique trial conditions with sequential constraints.

3. Generated randomized lists to use in a recognition experiment.


# Assignment

* Write code in a Jupyter notebook (after making a copy and renaming it to have your userid in the title --- e.g., A02_Memory_ListGen_mst3k).

## Design

Your assignment is to write a script that creates lists of dictionaries that you will later present to participants as part of an experiment.  

The script should be configurable such that you can specify different
numbers of lists and trials, along with other details specific to the
experiment you decide to do.

Each dictionary represents a trial and should contain all the
information necessary to identify the stimulus to be presented,
details about that stimulus, and the condition in which to present it.
This information will be experiment-specific, as outlined below.

You have two options for your experiment.  Please select **one** of
the following experiments, keeping in mind that your next assignment
will be to code the experiment presentation and response collection
for the lists you generate from this assignment.
  
* ***When you are done, make sure you have run every cell, so that we can see it ran without error and produces the correct output. Then please save the notebook as HTML (`File -> Download as -> HTML`) and upload it to the matching assignment on Canvas.***  

## Option 1: Refreshing Valence Study

The main question of this study is whether recognition memory for
words depends on the emotional or affective valence of those words and whether there is an interaction between attention refreshing and valence.

Participants will study lists of positive (+), negative (-), and
neutral (~) words and then, after a short delay, they will be given a
recognition test over all the studied target words plus a matched set
of non-studied lures.  The stimuli are contained in three separate CSV
files:

- [Positive Pool](./pos_pool.csv)
- [Negative Pool](./neg_pool.csv)
- [Neutral Pool](./neu_pool.csv)

You will need to read these files in as lists of dictionaries (hint,
use the ``DictReader`` from the ``csv`` module that was covered in
class.)  

Use these pools to create lists with trials of valence crossed with three experimental conditions:

1. *Repeated*: Where a word will be immediately repeated as the next word.
2. *Refreshed*: Where you will indicate the participant should "refresh" the previous word by presenting a "+".
3. *Once-presented*: Where a word is only presented once and is repeated or refreshed.

We suggest that you generate the study items for a list in two stages. In the first stage you shuffle all combinations of the trial types (Valence crossed with Condition). In the second stage you loop over those conditions and append trials to a block depending on the information in each trial type. For the Repeated and Refreshed you would append two items, for the Once-presented you would only append one.

You will need to generate a matching test list for each study list
that includes all the studied items, plus a set of lures that match
the valence of the studied words.

Be sure to add in information to each trial dictionary that identifies
the word, its valence, the condition of that trial, and whether it is a
target or a lure.  Feel free to add in more information if you would
like.

## Option 2: Spacing Scene Study

This study will test whether recognition memory for outdoor and outdoor
scenes is modulated by whether the items are once-presented, repeated immediately following the first presentation of the item (i.e., massed repetition), or repeated after a number of other items (i.e., spaced repetition). The participants will then be given a
recognition test over all the studied target images plus a matched set
of non-studied lures.  You can access the lists of stimuli available:

- [outdoor Pool](./indoor.csv)
- [Outdoor Pool](./outdoor.csv)

You will need to read these files in as lists of dictionaries (hint,
use the ``DictReader`` from the ``csv`` module that was covered in
class.)  For the actual experiment we will give you the images that
are referenced by the file names in these pools, but for the list
generation you do not need the images, themselves and should identify
the image you will be presenting using the file name.  

Use these pools to create lists of trials for the experimental conditions consiting of indoor/outdoor vs once-presented/massed/spaced items. Each
list should contain an equal number of each combination of these conditions in *random* order, but handling the spaced items will take some care. 

While the massed items come immediately after the first time the item was presented, the spaced repetitions need to come at a range of 3 to 7 (though this should be a configuration variable) items following the first presentation of the matching item. We will provide some suggestions for how to attain this structure in class discussions, but generally following a two-stage approach of shuffling all possible conditions first and then filling in specific items will work best. *Note, you can not have a spaced item condition in the last two slots on the list because it would not be possible to have the repetition be spaced.*

You will need to generate a matching test list for each study list
that includes all the studied items, plus a set of lures that match
the image categories from the studied items.

Be sure to add in information to each trial dictionary that identifies
the file name, the category of the image, the condition of the trial,
and whether it is a target or a lure.


# My work

## Outline and test code

Study phase
- Should be randomized indoor outdoor
- Conditions: 
    - How to balance number of items in each condition (item specific), vs number of instances of each condition (item agnostic)
        - These will be generated for each condition
    - Once presented
        - simple just present the item once
        - do this one last cause it can go anywhere
        - e.g. `{"pool":"indoor", "type":"1p", "reps":0, "distances":[None], "placement":[0]}`
    - massed repeated
        - take up rep number of slots 
        - all slots are contiguous
        - should have param for distribution of number of repetitions=default to one level
        - e.g. `{"pool":"indoor", "type":"massed-rep", "reps":3, "distances":[1,1], "placement":[0,1,2]}`
    - spaced repeated
        - take up rep number of slots
        - slots are not continuous
        - should have param for distribution of number of repetitions=default to one level
        - should have param for distance of repetitions=default to one level
        - e.g. `{"pool":"indoor", "type":"spaced-rep", "reps":3, "distances":[3,3], "placement":[0,3,6]}`

Test phase
- Should be randomized between indoor outdoor
- Should have varied time from study test (ie early middle late items)
- 2fac or old/new?

Check my code
- summary stats
    - number of each condition (in/out)
    - number of condition items (1p, massed rep, spaced rep)
    - number of condition trials (1p, massed rep, spaced rep)
- for each trial check numbers add up (ie image is used 3 times if it said so)


In [1]:
import numpy as np  # for classic array stuff
import csv          # for reading in files
import random       # for shuffling lists
import pandas as pd # for tables
import logging      # practice doing good stuff
from copy import deepcopy # for fixing terrible bugs

logging_level = logging.WARNING
logging.basicConfig(format='%(levelname)s (%(asctime)s): %(message)s (Line: %(lineno)d [%(filename)s])',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging_level)

In [38]:
# Make the set of possible conditions
# Output should be list of dictionaries like the examples below
# {"pool":"indoor", "type":"1p", "reps":0, "distances":[None], "placement":[0]}
# {"pool":"indoor", "type":"massed-rep", "reps":3, "distances":[1,1], "placement":[0,1,2]}
# {"pool":"indoor", "type":"spaced-rep", "reps":3, "distances":[3,3], "placement":[0,3,6]}

POOLS = ["indoor", "outdoor"]
CONDITION_TYPES = ["1p", "massed-rep", "spaced-rep"]
REP_TYPES = [2]
DISTANCE_TYPES = [np.arange(3,7),] # list of distributions. If single number then 

conditions = []
for pool in POOLS:
    for condition_type in CONDITION_TYPES:
        if condition_type=="1p":
            # do stuff
            conditions.append({"pool":pool, "type":condition_type, "reps":1, "distances":None, "placement":np.array([0])})
        else:
            for reps in REP_TYPES:
                placements = np.arange(reps)
                if condition_type=="massed-rep":
                    distances = np.diff(placements)
                    conditions.append({"pool":pool, "type":condition_type, "reps":reps, "distances":distances, "placement":placements})

                elif condition_type=="spaced-rep":
                    # If a list then it picks a random one. 
                    for dist in DISTANCE_TYPES:
                        distances = dist.copy()
                        conditions.append({"pool":pool, "type":condition_type, "reps":reps, "distances":distances, "placement":placements})

print(f"There are {len(conditions)} conditions")
for c in conditions:
    print(c)

There are 6 conditions
{'pool': 'indoor', 'type': '1p', 'reps': 1, 'distances': None, 'placement': array([0])}
{'pool': 'indoor', 'type': 'massed-rep', 'reps': 2, 'distances': array([1]), 'placement': array([0, 1])}
{'pool': 'indoor', 'type': 'spaced-rep', 'reps': 2, 'distances': array([4]), 'placement': array([0, 4])}
{'pool': 'outdoor', 'type': '1p', 'reps': 1, 'distances': None, 'placement': array([0])}
{'pool': 'outdoor', 'type': 'massed-rep', 'reps': 2, 'distances': array([1]), 'placement': array([0, 1])}
{'pool': 'outdoor', 'type': 'spaced-rep', 'reps': 2, 'distances': array([4]), 'placement': array([0, 4])}


In [37]:
random.choice(np.arange(0,4))

0

In [16]:
# Generate trial_set for the experiment from conditions
# We are specifically adding repetions of conditions and image filenames

# First get all the images
# create a dictionary reader
indoor_reader = csv.DictReader(open('indoor.csv','r'))
# read in all the lines into a list of dicts
images_indoor = [l for l in indoor_reader]
# create a dictionary reader
outdoor_reader = csv.DictReader(open('outdoor.csv','r'))
# read in all the lines into a list of dicts
images_outdoor = [l for l in outdoor_reader]

# shuffle the images
random.shuffle(images_indoor)
random.shuffle(images_outdoor)

# Figure out the number of times we can repeat all the conditions evenly with the given images 
indoor_reps = int(np.floor(len(images_indoor)/(len(conditions)/2)))
outdoor_reps = int(np.floor(len(images_outdoor)/(len(conditions)/2)))
condition_reps = np.min((indoor_reps, outdoor_reps))

# initialize the trial set
trial_set = []
for i in range(condition_reps):
    for condition in conditions:
        trial_set.append(condition.copy())
# TODO this is where I would add a trial length changes. Could be with or with out full counterbalancing
 


In [17]:
# add id, image file names, and spacing
for i, trial in enumerate(trial_set):
    trial['id'] = i
    
    if trial["pool"]=="indoor":
        trial["image_filename"] = images_indoor[0]["filename"]
        images_indoor.pop(0)
    elif trial["pool"]=="outdoor":
        trial["image_filename"] = images_outdoor[0]["filename"]
        images_outdoor.pop(0)
    else:
        print("invalid pool type")

    if trial['type']=='spaced-rep':
        
        distances = np.diff(placements)

# trial_set now has file names for each condition
print(f"There are {len(trial_set)} trials")
for trial in trial_set:
    print(trial)

There are 618 trials
{'pool': 'indoor', 'type': '1p', 'reps': 1, 'distances': None, 'placement': array([0]), 'id': 0, 'image_filename': 'in0306.jpg'}
{'pool': 'indoor', 'type': 'massed-rep', 'reps': 2, 'distances': array([1]), 'placement': array([0, 1]), 'id': 1, 'image_filename': 'in0246.jpg'}
{'pool': 'indoor', 'type': 'spaced-rep', 'reps': 2, 'distances': array([5]), 'placement': array([0, 5]), 'id': 2, 'image_filename': 'in0311.jpg'}
{'pool': 'outdoor', 'type': '1p', 'reps': 1, 'distances': None, 'placement': array([0]), 'id': 3, 'image_filename': 'out0099.jpg'}
{'pool': 'outdoor', 'type': 'massed-rep', 'reps': 2, 'distances': array([1]), 'placement': array([0, 1]), 'id': 4, 'image_filename': 'out0060_new.jpg'}
{'pool': 'outdoor', 'type': 'spaced-rep', 'reps': 2, 'distances': array([5]), 'placement': array([0, 5]), 'id': 5, 'image_filename': 'out2302.jpg'}
{'pool': 'indoor', 'type': '1p', 'reps': 1, 'distances': None, 'placement': array([0]), 'id': 6, 'image_filename': 'in0405.jpg'

In [18]:
def check_placement(working_list, trial, placement):
    """
    Check if the placement fit for all repeats 
    """
    locations = trial['placement']+placement
    
    for location in locations:
        try:
            if not working_list[location]==None:
                # if there is no spot at this location, then this trial doesn't work here
                return False
        except IndexError:
            # if this location doesn't exist then this trial doesn't work
            return False
    # only if all locations fit is it true
    return True

def find_placement(working_list, trial):
    """
    Go through all possible remaining spots and propose placements
    Randomizes indice tried
    """
    proposal_indices = [index for index,value in enumerate(working_list) if value == None]
    good_inds = [] # list to store which of the proposal indices work for this trial
    for proposal in proposal_indices:
        if check_placement(working_list,trial=trial, placement=proposal):
            good_inds.append(proposal)    

    return good_inds

def place_trial_in_list(working_list, trial, proposal):
    
    for i, placement in enumerate(trial['placement']):
        working_list[proposal+placement] = trial.copy()
        working_list[proposal+placement]['repetition'] = i

    return working_list
        
def fit_trials_in_list(working_list, trial_set, level=0):
    """
    Here is a recursive thing
    The indexing into trial set could be a problem due to one trial could have multiple proposals 
        BUT I fit all the 1p last so those should be 1 to 1 with proposal inds
    """

    logging.debug(f"starting loops with {len(trial_set)} trials")
    if len(trial_set)<1: return working_list
    proposal_inds = find_placement(working_list, trial_set[0])
    logging.debug(f"starting loops with {len(proposal_inds)} proposals")
    
    if len(proposal_inds)>1:
        random.shuffle(proposal_inds) # don't want to try the trials in order

        temp_list = working_list.copy()
        for proposal in proposal_inds:
            # attempt a fit
            place_trial_in_list(temp_list, trial_set[0], proposal)
            # Check if that fit works for the rest of the trials
            temp_list = fit_trials_in_list(temp_list, trial_set[1:], level=level+1)
            if temp_list: 
                return temp_list # it works so let's use it
            else: 
                temp_list = working_list.copy() # it doesn't work so keep going
        
        # this should only happen if all proposals don't work for future trials
        return None
    
    elif len(proposal_inds)==1:
        temp_list = working_list.copy()
        # attempt a fit
        place_trial_in_list(temp_list, trial_set[0], proposal_inds[0])
        
        # Check if that fit works for the rest of the trials
        # But only if there are future trials
        if len(trial_set)>1:
            temp_list = fit_trials_in_list(temp_list, trial_set[1:], level=level+1)
            if temp_list: 
                # it works so let's use it
                return temp_list 
            else: 
                # no good future fits
                return None 
        # Only one trial left == nearly done!    
        elif len(trial_set)==1:
            logging.debug("Last trial placed!")
            return temp_list

    elif len(proposal_inds)<1:
        # only happens if there are trials left (otherwise it would have returned in the ==1 condition)
        logging.warning(f"No good proposals; recursion level {level}")
        return None
    
def complete_list_gen(trial_set):
    """ 
    Go through all trials and fit them into a spot
    """ 
    # first create null list
    trial_df = pd.DataFrame(trial_set)
    null_list = [None] * trial_df["reps"].sum()
    logging.debug(f"Null list is {len(null_list)} long")

    # Sort the trial set to be spaced and massed first then 1p
    trial_df = trial_df.sample(frac=1) # randomize the trials
    df_1p = trial_df[trial_df["type"]=='1p'] # get all the 1p trials
    df_massed_spaced = trial_df[trial_df["type"]!='1p'] # get all the massed/spaced trials
    sorted_trials = df_massed_spaced.to_dict('records') # convert the dataframe to a list with the massed/spaced trials first
    sorted_trials += df_1p.to_dict('records') # add the 1p trials to the end
    logging.debug(f"sorted trials is {len(sorted_trials)} long")
    
    # run the fitting process
    final_list = fit_trials_in_list(null_list, sorted_trials)

    return final_list

In [19]:
list_generated = complete_list_gen(trial_set)

In [20]:
for trial in list_generated:
    print(trial)

{'pool': 'indoor', 'type': '1p', 'reps': 1, 'distances': None, 'placement': array([0]), 'id': 456, 'image_filename': 'in0021.jpg', 'repetition': 0}
{'pool': 'indoor', 'type': '1p', 'reps': 1, 'distances': None, 'placement': array([0]), 'id': 66, 'image_filename': 'in0043.jpg', 'repetition': 0}
{'pool': 'outdoor', 'type': 'massed-rep', 'reps': 2, 'distances': array([1]), 'placement': array([0, 1]), 'id': 490, 'image_filename': 'out0140_new.jpg', 'repetition': 0}
{'pool': 'outdoor', 'type': 'massed-rep', 'reps': 2, 'distances': array([1]), 'placement': array([0, 1]), 'id': 490, 'image_filename': 'out0140_new.jpg', 'repetition': 1}
{'pool': 'outdoor', 'type': '1p', 'reps': 1, 'distances': None, 'placement': array([0]), 'id': 381, 'image_filename': 'out1589.jpg', 'repetition': 0}
{'pool': 'indoor', 'type': 'massed-rep', 'reps': 2, 'distances': array([1]), 'placement': array([0, 1]), 'id': 301, 'image_filename': 'in0295.jpg', 'repetition': 0}
{'pool': 'indoor', 'type': 'massed-rep', 'reps':